In [1]:
import pandas as pd
import requests
import base64
import json

In [2]:
with open('../accounts.json') as json_file:  
    data = json.load(json_file)

In [3]:
apikey = data['accounts']['Idealista']['apikey']
secret = data['accounts']['Idealista']['secret']
access_token = ''
token_type = ''

In [4]:
encoded_string = apikey + ":" + secret
encoded = base64.b64encode(encoded_string.encode())

In [5]:
authorization_header = 'Authorization'
grant_type = 'grant_type'
client_credentials = 'client_credentials'

In [6]:
r = requests.post('http://api.idealista.com/oauth/token',
             headers={authorization_header: 'Basic ' + encoded.decode()},
             params={grant_type:client_credentials})

In [7]:
access_token = r.json()['access_token']
token_type = r.json()['token_type']
search_authorization_value = token_type + ' ' + access_token

In [49]:
country = 'es'
operation = 'rent'
propertyType = 'homes' # 'bedrooms' is other option
center = '41.38621,2.15487', 
locale = 'en'
distance = 1300
locationId = '0-EU-ES-28' # ver qu es esto
maxItems = 50 # 50 as maximun allowed
maxPrice = 1000



In [50]:
params = {
    'locale':locale,
    'maxItems':maxItems,
    'operation':operation,
    'propertyType':propertyType,
    'apikey':apikey,
    'center' :center,
    'distance' :distance
}

In [51]:
home_search = requests.post('https://api.idealista.com/3.5/es/search',
                      headers={authorization_header: search_authorization_value},
                      params=params)

In [52]:
params[propertyType] = 'bedrooms'
beddrooms_search = requests.post('https://api.idealista.com/3.5/es/search',
                      headers={authorization_header: search_authorization_value},
                      params=params)

In [53]:
price = 'price'
propertyType = 'propertyType'
operation = 'operation'
rooms = 'rooms'
neighborhood = 'neighborhood'
latitude = 'latitude'
longitude = 'longitude'
distance = 'distance'
priceByArea = 'priceByArea'

In [ ]:
home_search.json()

In [55]:
homes = home_search.json().get('elementList')
bedrooms = beddrooms_search.json().get('elementList')

In [56]:
def add_item_to_dict(home, neighborhoods):
    item = { price:home[price],
             propertyType:home[propertyType],
             price:home[price],
             operation:home[operation],
             rooms:home[rooms],
             latitude:home[latitude],
             longitude:home[longitude],
             distance:home[distance],
             priceByArea:home[priceByArea]
            }

    if home[neighborhood] not in neighborhoods:
        neighborhoods[home[neighborhood]] = []
        
    neighborhoods[home[neighborhood]].append(item)
    

In [59]:
neighborhoods = {}
for home in homes:
    add_item_to_dict(home, neighborhoods)
for bedroom in bedrooms:
    add_item_to_dict(bedroom, neighborhoods)
        
    

In [64]:
def price_of_neighborhoods():
    neighs_price = {}
    for key in neighborhoods.keys():
        neighs_price[key] = price_per_neighborhood_homes(key)
    return neighs_price
            

In [65]:
def price_per_neighborhood(key):
    price_per_neighborhood = 0
    total_number_of_rooms = 0
    for appartment in neighborhoods[key]:
        if appartment[propertyType] == 'room':
            price_per_neighborhood += appartment[price]
            total_number_of_rooms += 1
        else:
            number_of_rooms = appartment[rooms] if appartment[rooms] > 0 else 1
            price_per_neighborhood += appartment[price] / number_of_rooms
            total_number_of_rooms += appartment[rooms]
    return (price_per_neighborhood / len(neighborhoods[key]), total_number_of_rooms)

In [66]:
price_of_neighborhoods()

{'El Raval': (941.25, 20),
 'Vila de Gràcia': (925.0, 10),
 'Sant Antoni': (516.5277777777778, 32),
 "L'Antiga Esquerra de l'Eixample": (607.6388888888888, 66),
 "La Nova Esquerra de l'Eixample": (458.452380952381, 82),
 'El Gòtic': (950.0, 2),
 "La Dreta de l'Eixample": (1272.5, 24),
 'Sant Gervasi - Galvany': (1383.3333333333333, 10)}

In [68]:
neighborhoods['El Gòtic']

[{'price': 950.0,
  'propertyType': 'flat',
  'operation': 'rent',
  'rooms': 1,
  'latitude': 41.3855032,
  'longitude': 2.1702249,
  'distance': '1283',
  'priceByArea': 21.0},
 {'price': 950.0,
  'propertyType': 'flat',
  'operation': 'rent',
  'rooms': 1,
  'latitude': 41.3855032,
  'longitude': 2.1702249,
  'distance': '1283',
  'priceByArea': 21.0}]

In [1]:
from Idealista import Idealista

In [2]:
ide = Idealista()

In [3]:
ide.create()

In [4]:
ide.get_neighborhoods(center='41.38621,2.15487',distance=1500)

41.38621,2.15487


TypeError: 'NoneType' object is not iterable